In [1]:
import torch
from dataset_utils import edge_stratified_split
from localtest.classification_dataset_test import ClassificationDataset
from  config.parser_config import config_parser
from torch.utils.data import DataLoader

/home/student/anaconda3/envs/efficientnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# GET EDGE DATA

In [32]:
parser = config_parser()
argv = '--config localtest/localtest_config'.split(' ') 
args  = parser.parse_args(argv)
full_dataset = ClassificationDataset(
        one_hot = False,
        augmentation= None,
        npz_path= args.npz_path,
        image_path= args.image_path,
        label_path= args.label_path,
        size = args.size,
        normalize= True)

edge_classes = ["Gryllteiste","Schnatterente","Buchfink","unbestimmte Larusmöwe",
                    "Schmarotzer/Spatel/Falkenraubmöwe","Brandgans","Wasserlinie mit Großalgen",
                    "Feldlerche","Schmarotzerraubmöwe","Grosser Brachvogel","unbestimmte Raubmöwe",
                    "Turmfalke","Trauerseeschwalbe","unbestimmter Schwan",
                    "Sperber","Kiebitzregenpfeifer",
                    "Skua","Graugans","unbestimmte Krähe"]

edge_labels = [full_dataset._get_label_from_cat(cat) for cat in edge_classes]

edge_train_data, edge_test_data = edge_stratified_split(full_dataset, full_labels = full_dataset._labels, edge_labels = edge_labels,  fraction = 0.8, random_state = 0)                     
trainloader = DataLoader(edge_train_data,
                                batch_size=1, 
                                shuffle=True,
                                num_workers=args.num_workers)

testloader = DataLoader(edge_test_data,
                                batch_size=1, 
                                shuffle=True,
                                num_workers=args.num_workers)

In [37]:
train_dict = {}
for _, label, cat, name in trainloader:
    cat = cat[0]
    name = name[0]
    if cat in train_dict.keys():
        train_dict[cat].append(name)
    else:
        train_dict[cat] = [name]
        

test_dict = {}
for _, label, cat, name in testloader:
    cat = cat[0]
    name = name[0]
    if cat in test_dict.keys():
        test_dict[cat].append(name)
    else:
        test_dict[cat] = [name]

In [38]:
to_dir = '/home/student/Dokumente/edge'

In [39]:
import os
import shutil 
for cat in train_dict.keys():
    for image in train_dict[cat]:
        from_loc = os.path.join(args.image_path, image)        
        train_dir = os.path.join(to_dir, cat, 'train')
        if not os.path.exists(train_dir):
            os.makedirs(train_dir)
        to_loc = os.path.join(train_dir, image)
        shutil.copy(from_loc, train_dir)
        

In [40]:
for cat in test_dict.keys():
    for image in test_dict[cat]:
        from_loc = os.path.join(args.image_path, image)        
        train_dir = os.path.join(to_dir, cat, 'test')
        if not os.path.exists(train_dir):
            os.makedirs(train_dir)
        to_loc = os.path.join(train_dir, image)
        shutil.copy(from_loc, train_dir)

# GAN

In [55]:
from model.vanilla_cgan import Generator
import torch.nn as nn
device = torch.device('cpu')
map_location=torch.device('cpu')
class_num = 118
img_size = 256
model_dim = 256
lr = 0.00005
batch_size = 25
z_size = 1
generator_layer_size = [model_dim, model_dim*2, model_dim*4]
generator = Generator(generator_layer_size, z_size, img_size, class_num)
# discriminator = Discriminator(discriminator_layer_size, img_size, class_num).to(device)
criterion = nn.BCELoss()
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)
# d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)

In [56]:
map_location=torch.device('cpu')
g_checkpoint= torch.load('saved_models/vanilla_gan/generator_TRAIN_cGAN,_dim:256,_lr:_5e-05,_epochs:_10000,_size:_256.pt', map_location=torch.device('cpu'))
generator.load_state_dict(g_checkpoint['model_state_dict'])
g_optimizer.load_state_dict(g_checkpoint['optimizer_state_dict'])
epoch = g_checkpoint['epoch']
g_loss = g_checkpoint['loss']

RuntimeError: Error(s) in loading state_dict for Generator:
	size mismatch for model.0.weight: copying a param with shape torch.Size([1024, 119]) from checkpoint, the shape in current model is torch.Size([256, 119]).
	size mismatch for model.0.bias: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for model.2.weight: copying a param with shape torch.Size([512, 1024]) from checkpoint, the shape in current model is torch.Size([512, 256]).
	size mismatch for model.4.weight: copying a param with shape torch.Size([256, 512]) from checkpoint, the shape in current model is torch.Size([1024, 512]).
	size mismatch for model.4.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for model.6.weight: copying a param with shape torch.Size([196608, 256]) from checkpoint, the shape in current model is torch.Size([196608, 1024]).